# Prep filtered scaffold sets for distributed design

### Imports

In [1]:
# %load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
# %matplotlib inline
# # reloads modules automatically before executing cells
# %load_ext autoreload
# %autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /home/broerman/crispy_shifty/projects/OPS/round_1/design
running on node: sofia


In [ ]:
# fixed messed up inputs
pyrosetta.init()
pose = pyrosetta.pose_from_pdb("/home/broerman/crispy_shifty/projects/OPS/round_1/design/00_scaffold_sets/inputs/input_switched/R6CP33_m1.pdb")
sw = pyrosetta.rosetta.protocols.simple_moves.SwitchChainOrderMover()
sw.chain_order("21")
sw.apply(pose)
pose.dump_pdb("/home/broerman/crispy_shifty/projects/OPS/round_1/design/00_scaffold_sets/inputs/R6CP33_m1.pdb")
pose = pyrosetta.pose_from_pdb("/home/broerman/crispy_shifty/projects/round_1/design/OPS/00_scaffold_sets/inputs/input_switched/DLPx6_PDL_0_4_5.pdb")
pdb_info = pyrosetta.rosetta.core.pose.PDBInfo(pose)
pose.pdb_info(pdb_info)
pose.dump_pdb("/home/broerman/crispy_shifty/projects/OPS/round_1/design/00_scaffold_sets/inputs/DLPx6_PDL_0_4_5.pdb")

### Domesticate the scaffolds by trimming off leading and trailing loops and adding metadata to the output pdb.bz2s. 

In [2]:
design_list_file = "/home/broerman/crispy_shifty/projects/OPS/round_1/design/00_scaffold_sets/inputs.list"

with open(design_list_file, "w") as f:
    f.write("\n".join(sorted(glob("/home/broerman/crispy_shifty/projects/OPS/round_1/design/00_scaffold_sets/inputs/*.pdb"))))

In [3]:
sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "01_prep_inputs"
design_list_file = "/home/broerman/crispy_shifty/projects/OPS/round_1/design/00_scaffold_sets/inputs.list"
output_path = os.path.join("/home/broerman/crispy_shifty/projects/OPS/round_1/design", f"{simulation_name}")

options = " ".join(
    [
        "out:level 200",
        "corrections:beta_nov16 true",
        "holes:dalphaball /software/rosetta/DAlphaBall.gcc",
        "run:preserve_header true" # for "Tomponents"
    ]
)
extra_kwargs = {
    "metadata_csv": "/home/broerman/crispy_shifty/projects/OPS/round_1/design/00_scaffold_sets/metadata.csv",
    "chains_to_keep": "12",
    "num_ss_per_repeat": "2",
    "fixed_resis": "distribute"
}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.cleaning.add_metadata_to_input",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="short",
    memory="2G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

Run the following command with your desired environment active:
sbatch -a 1-26 /home/broerman/crispy_shifty/projects/OPS/01_prep_inputs/run.sh


### Collect scorefiles of the prepped input scaffolds

In [3]:
sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "01_prep_inputs"
output_path = os.path.join("/home/broerman/crispy_shifty/projects/OPS/round_1/design", f"{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile

In [4]:
sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import parse_scorefile_linear

output_path = os.path.join("/home/broerman/crispy_shifty/projects/OPS/round_1/design", f"{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))

  0%|          | 0/34 [00:00<?, ?it/s]

### Dump scores_df as a CSV and then reload, for performance reasons

In [5]:
simulation_name = "01_prep_inputs"
output_path = os.path.join("/home/broerman/crispy_shifty/projects/OPS/round_1/design", f"{simulation_name}")
if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df.to_csv(os.path.join(output_path, "scores.csv"))

scores_df = pd.read_csv(os.path.join(output_path, "scores.csv"), index_col="Unnamed: 0")

### Save a list of outputs

In [6]:
# this design has binding functionality in the loops which will require significant modification to existing protocols to hingify
excluded_inputs = set(["DLPx6_PDL_0_4_5"])

with open(
    "01_prep_inputs/prepped_inputs.list", "w"
) as f:
    for path, row in scores_df.iterrows():
        if row['pdb'].split('/')[-1].split('.')[0] not in excluded_inputs:
            print(path, file=f)

In [7]:
# check fixed resis in pymol
for index, row in scores_df.iterrows():
    name = index.split('/')[-1].split('.')[0]
    # print(f"select {row['pdb'].split('/')[-1].split('.')[0]}, {name} and resid {'+'.join(row['fixed_resis'].split(','))};")
    print(f"color green, {name} and resid {'+'.join(row['fixed_resis'].split(','))};")

color green, 01_prep_inputs_a2a3f7d44d8344de953b158c18a98224 and resid 10+11+14+15+17+18+19+53+54+57+58+60+61+62+96+97+100+101+103+104+105+139+140+143+144+146+147+148+182+183+186+187+189+190+191+225+226+229+230+232+233+234;
color green, 01_prep_inputs_08790e32144d489e86d77e343750b47b and resid 3+6+7+10+11+14+18+45+48+49+52+53+56+60+87+90+91+94+95+98+102+129+132+133+136+137+140+144;
color green, 01_prep_inputs_445277da1b4f420d82945280feac6379 and resid 6+7+10+11+15+19+53+54+57+58+62+66+100+101+104+105+109+113+147+148+151+152+156+160+194+195+198+199+203+207+242+246+250+254;
color green, 01_prep_inputs_378678039e684559baeb629273e48963 and resid 28+29+32+35+36+39+71+72+75+78+79+82+114+115+118+121+122+125+157+158+161+164+165+168;
color green, 01_prep_inputs_405f5a0b5b1648249c6a5f941b95b5b1 and resid 43+46+50+98+101+105+153+156+160+208+211+215;
color green, 01_prep_inputs_0226fbee4dbd472ca9baa3805917ba93 and resid 5+8+12+15+16+19+20+22+60+63+67+70+71+74+75+77+115+118+122+125+126+129+130+132+